In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
torch.cuda.empty_cache()

# **Install dependencies**

In [ ]:
%%capture
!pip install tensorflow
!pip install keras
!pip install netron
!pip install nltk

!pip install git+https://github.com/huggingface/transformers
!pip install datasets
!pip install nlp
!pip install rouge-score
#!pip install sacrebleu
!pip install git-python
!pip install sentencepiece
#!pip install torchdata
#!pip install transformers
!pip install sacrebleu==1.5.1
!pip install tree-sitter
!pip install evaluate

# **Import Libraries**

In [ ]:
import transformers
import datasets
from datasets import load_dataset, load_metric
import logging
from transformers import BertTokenizer, GPT2Tokenizer, GPT2TokenizerFast, EncoderDecoderModel, Trainer, TrainingArguments, BertTokenizerFast
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

logging.basicConfig(level=logging.INFO)

import types
import argparse
import logging
from functools import partial
import json

import torch
from torch import optim
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import (
    BertGenerationConfig,
    BertGenerationEncoder,
    BertTokenizer,
    EncoderDecoderModel,
    EncoderDecoderConfig,
    GPT2LMHeadModel,
    GPT2TokenizerFast,
    Trainer,
    TrainingArguments,
    get_cosine_schedule_with_warmup,
)

import sacrebleu
import random
import numpy as np
import pandas as pd
from datasets import Dataset
#from tensorboardX import SummaryWriter


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
%cd /content/drive/MyDrive/models/CodeGeneration/MarianCG-CoNaLa

/content/drive/MyDrive/models/CodeGeneration/MarianCG-CoNaLa


# **Another way to load data from huggingface**

In [ ]:
from datasets import load_dataset

raw_dataset = load_dataset("AhmedSSoliman/CoNaLa")

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
train_dataset= raw_dataset["train"]
val_dataset = raw_dataset["validation"]
test_dataset = raw_dataset["test"]

In [ ]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['intent', 'snippet'],
        num_rows: 11125
    })
    test: Dataset({
        features: ['intent', 'snippet'],
        num_rows: 500
    })
    validation: Dataset({
        features: ['intent', 'snippet'],
        num_rows: 1237
    })
})

# **Prepare the data.**

In [ ]:
encoder_length = 32
decoder_length = 32
batch_size = 1


# map data correctly
def map_to_encoder_decoder_inputs(batch):    
    inputs = tokenizer(batch["intent"], padding="max_length", truncation=True, max_length=encoder_length)
    outputs = tokenizer(batch["snippet"], padding="max_length", truncation=True, max_length=decoder_length)
    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask
    batch["decoder_input_ids"] = outputs.input_ids
    batch["labels"] = outputs.input_ids.copy()
    batch["decoder_attention_mask"] = outputs.attention_mask
    
    """
    # complicated list comprehension here because pad_token_id alone is not good enough to know whether label should be excluded or not
    batch["labels"] = [
        [-100 if mask == 0 else token for mask, token in mask_and_tokens] for mask_and_tokens in [zip(masks, labels) for masks, labels in zip(batch["decoder_attention_mask"], batch["labels"])]
    ]
    """
    assert all([len(x) == encoder_length for x in inputs.input_ids])
    assert all([len(x) == decoder_length for x in outputs.input_ids])
    
    return batch

In [ ]:
# make train dataset ready
train_data = train_dataset.map(
    map_to_encoder_decoder_inputs, batched=True, batch_size=1, remove_columns=['intent', 'snippet'],
)
train_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)



  0%|          | 0/11125 [00:00<?, ?ba/s]

In [ ]:
# same for validation dataset
val_data = val_dataset.map(
    map_to_encoder_decoder_inputs, batched=True, batch_size=1, remove_columns=['intent', 'snippet'],
)
val_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)


  0%|          | 0/1237 [00:00<?, ?ba/s]

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer,
                                       max_length=512,padding=True, ####new
                                       model = model)

# **Training**

In [ ]:
# set training arguments - these params are not really tuned, feel free to change
training_args = Seq2SeqTrainingArguments(
    output_dir="./Marian-Training",
    evaluation_strategy="epoch",
    save_strategy = "epoch",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    predict_with_generate=True,
    num_train_epochs=10,
    do_train=True,
    do_eval=True,
    fp16=True,
    overwrite_output_dir=True,
    learning_rate = 1e-5,
    weight_decay=0.01, 
    warmup_ratio = 0.05,
    seed = 1995,
    save_total_limit = 2,
    load_best_model_at_end = True,
    #push_to_hub=True,
)
#    

# instantiate trainer
trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    compute_metrics=evaluator,
    data_collator = data_collator,
    train_dataset=train_data,
    eval_dataset =val_data,
       
)
#    save_total_limit=3,
# 

Using cuda_amp half precision backend


In [ ]:
#Marian-CoNaLa
trainer.train()

In [ ]:
trainer.save_model()
trainer.save_state()

Saving model checkpoint to ./Marian-Training
Configuration saved in ./Marian-Training/config.json
Model weights saved in ./Marian-Training/pytorch_model.bin
tokenizer config file saved in ./Marian-Training/tokenizer_config.json
Special tokens file saved in ./Marian-Training/special_tokens_map.json
